# Modèles 

In [15]:
import pandas as pd
import numpy as np

Dans cette partie on va essayer d'utiliser des modèles d'apprentissage machine pour prédire les scores des joueurs. On commence par récupérer le dataset et le mettre en forme pour l'apprentissage.

Les choix que l'on fait sont les suivants :
  - On ne garde pas les variables qui dépendent du match :
      - list_var_avg + WIN
  - On ne garde pas les variables qui nous donne aucune information : 
      - SEASON_ID, GAME_ID
  - On modifie le type de la variable PLAYER_ID pour pas qu'elle soit considérée comme un nombre
  - On ne garde pas les variables agrégées sur les équipes car elles sont trop souvent `Nan` car les équipes ne se jouent pas souvent (4 fois max)

In [9]:
# La liste des variables dont on va récupérer les moyennes
list_var_avg = ['PTS', 'REB', 'AST', 'STL', 'BLK', 'FGM', 'FTM', 'FG3M',
                 'TOV', 'FGF', 'FG3F', 'FTF', 'BONUS', 'MALUS', 'SCORE']

In [65]:
# Récupere le dataset
df = pd.read_csv('../data/boxscore_2019_final.csv')

df = df.drop(list_var_avg[:-1]+['WIN', 'SEASON_ID', 'GAME_ID'], axis=1)
df = df[~df.AVG_SCORE.isna()]

df.PLAYER_ID = df.PLAYER_ID.astype(object)

# Pour l'instant on retire aussi les variables de moyenne par équipe car elles 
# sont trop souvant vide
df = df.drop(['AVG_ADD_'+x for x in list_var_avg]+['AVG_BONUS', 'AVG_MALUS', 'AVG_SCORE'], axis=1)

In [66]:
df.sample(3)

,PLAYER_ID,GAME_DATE,TEAM,ADV,HOME,SCORE,AVG_PTS,AVG_REB,AVG_AST,AVG_STL,AVG_BLK,AVG_FGM,AVG_FTM,AVG_FG3M,AVG_TOV,AVG_FGF,AVG_FG3F,AVG_FTF,NB_VIC,NB_VIC_ADD
2399,201580,2019-10-29,LAL,MEM,True,21,4.666667,3.333333,0.333333,0.333333,0.666667,2.333333,0.0,0.000000,1.333333,1.666667,0.000000,0.333333,2,0
12716,1626209,2020-08-13,POR,BKN,False,-3,6.333333,5.000000,2.000000,1.000000,0.000000,2.333333,1.0,0.666667,0.333333,3.333333,2.333333,0.333333,2,0
2712,201599,2020-03-03,BKN,BOS,False,27,9.333333,11.000000,1.333333,0.000000,0.666667,3.666667,2.0,0.000000,0.666667,1.333333,0.000000,1.333333,0,1


## 1 - Séparation de la base

Pour pouvoir évaluer les différents modèles que nous allons utiliser Sélim nous a conseiller de faire une *K-fold cross validation* qui très grossièrement consiste à tester son modèles sur différentes partie du data set pour mesurer sa robustesse.

Dans le cas de données temporelles on ne peut pas simplement prendre des parties aléatoires car on risquerait d'avoir des données du futur dans notre base d'entrainement et des données du passé dans celle de test. On se base sur cet [article](https://medium.com/@soumyachess1496/cross-validation-in-time-series-566ae4981ce4) pour les différentes méthodes de *cross-validation* dans des séries temporelles.

Les deux méthodes simples que l'on retient sont :
  - *Time Series Split Cross-Validation*
  - *Blocked Cross-Validation*

IMAGE 1 // IMAGE 2

La deuxième est utilisé lorsque l'on suspecte que connaitre des données futurs peut permetre de prédire plus précisément des données passées. Comme nous partons du principe que la probabilité de faire un bon score ne dépend que du passé et jamais du futur nous avons décidé d'implémenter la méthode *Time Serie Split Cross-Validation*

In [67]:
# Le nombre de cross validation+1 que l'on veut
K=6

In [68]:
# On transforme les variables catégorielles en encodage oneshot
df = pd.get_dummies(df, columns=['PLAYER_ID', 'TEAM', 'ADV'])

# On récupère les dates pour les séparer en K parties
dates = df.GAME_DATE.sort_values().unique()

# On crée les critères de séléction la base
criteres = [df.GAME_DATE.between(x[0], x[-1]) for x in np.array_split(dates, K)]
valeurs = range(K)

# On créé la variable de découpe
df = df.assign(SPLIT=np.select(criteres, valeurs, df.GAME_DATE))

# On créé les dataframes
df.groupby('SPLIT').size().to_frame(name='SIZE')

,SIZE
SPLIT,
0,4358
1,4775
2,4653
3,4644
4,3211
5,853


On remarque que la dernière partie ne contient que très peu d'entrées. On l'explique par le fait que l'on a séparer les groupes en se basant sur les dates et non sur le nombre de match entre deux dates. À la fin de la saison on se retrouve avec les playoff donc il y a beaucoup moins de match. Pour équilibrer on va concatener les deux dernières parties.

In [69]:
lst_df = []

for _, df_split in df.drop('GAME_DATE', axis=1).groupby('SPLIT'):
    lst_df += [df_split]

last_df = lst_df.pop()
lst_df[-1] = pd.concat([lst_df[-1], last_df])

In [70]:
[len(x) for x in lst_df]

[4358, 4775, 4653, 4644, 4064]

On se retrouve donc avec K-1 bases de données assez équilibrées sur lesquelles on peut tester les modèles de machines learnings.

## 1 - Modèles linéaires

In [ ]:
# On ne garde que les joueurs qui ont joué plus de 50 matchs sur la saison
df_players = (df.groupby('PLAYER_ID').count() > 50)
list_players = df_players[df_players.SCORE].index
df = df[df.PLAYER_ID.isin(list_players)].reset_index(drop=True)